In [2]:
import numpy as np

class Tensor (object):
    
    def __init__(self, data):
        self.data = np.array(data)
    def __add__(self, other):
        return Tensor(self.data + other.data)
    def __repr__(self):
        return str(self.data.__repr__())
    def __str__(self):
        return str(self.data.__str__())

x = Tensor([1,2,3,4,5])
print(x)

y = x + x
print(y)

[1 2 3 4 5]
[ 2  4  6  8 10]


In [7]:
import numpy as np

class Tensor (object):
    
    def __init__(self, data, creators=None, creation_op=None):
        self.data = np.array(data)
        self.creation_op = creation_op
        self.creators = creators
        self.grad = None
    
    def backward(self, grad):
        self.grad = grad
        if(self.creation_op == "add"):
            self.creators[0].backward(grad)
            self.creators[1].backward(grad)
    
    def __add__(self, other):
        return Tensor(self.data + other.data, creators=[self,other], creation_op="add")

    def __repr__(self):
        return str(self.data.__repr__())

    def __str__(self):
        return str(self.data.__str__())

x = Tensor([1,2,3,4,5])
y = Tensor([2,2,2,2,2])
z = x + y
z.backward(Tensor(np.array([1,1,1,1,1])))

In [8]:
print(x.grad)
print(y.grad)
print(z.creators)
print(z.creation_op)

[1 1 1 1 1]
[1 1 1 1 1]
[array([1, 2, 3, 4, 5]), array([2, 2, 2, 2, 2])]
add


In [10]:
a = Tensor([1,2,3,4,5])
b = Tensor([2,2,2,2,2])
c = Tensor([5,4,3,2,1])
d = Tensor([-1,-2,-3,-4,-5])
e = a + b
f = c + d
g = e + f
g.backward(Tensor(np.array([1,1,1,1,1])))
print(a.grad)

[1 1 1 1 1]


In [11]:
a = Tensor([1,2,3,4,5])
b = Tensor([2,2,2,2,2])
c = Tensor([5,4,3,2,1])

d = a + b
e = b + c
f = d + e
f.backward(Tensor(np.array([1,1,1,1,1])))
print(b.grad.data == np.array([2,2,2,2,2]))

[False False False False False]


In [14]:
import numpy as np

class Tensor (object):
    def __init__(self,data, autograd=False, creators=None, creation_op=None, id=None):
        
        self.data = np.array(data)
        self.creators = creators
        self.creation_op = creation_op
        self.grad = None
        self.autograd = autograd
        self.children = {}
        if(id is None):   # テンソルの子の数を追跡
            id = np.random.randint(0, 100000)
        self.id = id
        
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1
                    
    # テンソルがそれぞれの子から正しい数の勾配を受け取っているかどうかを確認
    def all_children_grads_accounted_for(self):
        for id,cnt in self.children.items():
            if(cnt != 0):
                return False
        return True

    def backward(self, grad=None, grad_origin=None):
        if(self.autograd):
            if(grad_origin is not None):
                # 逆伝播できること,または勾配を待っているかどうかをチェックし,
                # その場合はカウンタを1減らす
                if(self.children[grad_origin.id] == 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1
                    
            if(self.grad is None):
                self.grad = grad    # 複数の子からの勾配を蓄積
            else:
                self.grad += grad
                
            if(self.creators is not None and (self.all_children_grads_accounted_for() or grad_origin is None)):
                
                if(self.creation_op == "add"):    # 実際の逆伝播を開始
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)
                    
    def __add__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                           autograd=True,
                           creators=[self,other],
                           creation_op="add")
        return Tensor(self.data + other.data)
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())

a = Tensor([1,2,3,4,5], autograd=True)
b = Tensor([2,2,2,2,2], autograd=True)
c = Tensor([5,4,3,2,1], autograd=True)
d = a + b
e = b + c
f = d + e
f.backward(Tensor(np.array([1,1,1,1,1])))
print(b.grad.data == np.array([2,2,2,2,2]))

[ True  True  True  True  True]


In [2]:
import numpy as np

class Tensor (object):
    def __init__(self,data, autograd=False, creators=None, creation_op=None, id=None):
        
        self.data = np.array(data)
        self.creators = creators
        self.creation_op = creation_op
        self.grad = None
        self.autograd = autograd
        self.children = {}
        if(id is None):   # テンソルの子の数を追跡
            id = np.random.randint(0, 100000)
        self.id = id
        
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1
                    
    # テンソルがそれぞれの子から正しい数の勾配を受け取っているかどうかを確認
    def all_children_grads_accounted_for(self):
        for id,cnt in self.children.items():
            if(cnt != 0):
                return False
        return True

    def backward(self, grad=None, grad_origin=None):
        if(self.autograd):
            if(grad_origin is not None):
                # 逆伝播できること,または勾配を待っているかどうかをチェックし,
                # その場合はカウンタを1減らす
                if(self.children[grad_origin.id] == 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1
                    
            if(self.grad is None):
                self.grad = grad    # 複数の子からの勾配を蓄積
            else:
                self.grad += grad
                
            if(self.creators is not None and (self.all_children_grads_accounted_for() or grad_origin is None)):
                
                if(self.creation_op == "add"):    # 実際の逆伝播を開始
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)

                if(self.creation_op == "neg"):
                    self.creators[0].backward(self.grad.__neg__())

    def __add__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                           autograd=True,
                           creators=[self,other],
                           creation_op="add")
        return Tensor(self.data + other.data)
    
    def __neg__(self):
        if(self.autograd):
            return Tensor(self.data * -1,
                           autograd=True,
                           creators=[self],
                           creation_op="neg")
        return Tensor(self.data * -1)
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())

a = Tensor([1,2,3,4,5], autograd=True)
b = Tensor([2,2,2,2,2], autograd=True)
c = Tensor([5,4,3,2,1], autograd=True)

d = a + (-b)
e = (-b) + c
f = d + e

f.backward(Tensor(np.array([1,1,1,1,1])))

print(b.grad.data == np.array([-2,-2,-2,-2,-2]))

[ True  True  True  True  True]


In [4]:
import numpy as np

class Tensor (object):
    def __init__(self,data, autograd=False, creators=None, creation_op=None, id=None):
        
        self.data = np.array(data)
        self.creators = creators
        self.creation_op = creation_op
        self.grad = None
        self.autograd = autograd
        self.children = {}
        if(id is None):   # テンソルの子の数を追跡
            id = np.random.randint(0, 100000)
        self.id = id
        
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1
                    
    # テンソルがそれぞれの子から正しい数の勾配を受け取っているかどうかを確認
    def all_children_grads_accounted_for(self):
        for id,cnt in self.children.items():
            if(cnt != 0):
                return False
        return True

    def backward(self, grad=None, grad_origin=None):
        if(self.autograd):
            if(grad_origin is not None):
                # 逆伝播できること,または勾配を待っているかどうかをチェックし,
                # その場合はカウンタを1減らす
                if(self.children[grad_origin.id] == 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1
                    
            if(self.grad is None):
                self.grad = grad    # 複数の子からの勾配を蓄積
            else:
                self.grad += grad
                
            if(self.creators is not None and (self.all_children_grads_accounted_for() or grad_origin is None)):
                
                if(self.creation_op == "add"):    # 実際の逆伝播を開始
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)
                
                if(self.creation_op == "sub"):
                    self.creators[0].backward(Tensor(self.grad.data), self)
                    self.creators[1].backward(Tensor(self.grad.__neg__().data), self)
                
                if(self.creation_op == "mul"):
                    new = self.grad * self.creators[1]
                    self.creators[0].backward(new, self)
                    new = self.grad * self.creators[0]
                    self.creators[1].backward(new, self)
                    
                if(self.creation_op == "mm"):
                    act = self.creators[0]
                    weights = self.creators[1]
                    new = self.grad.mm(weights.transpose())
                    act.backward(new)
                    new = self.grad.transpose().mm(act).transpose()
                    weights.backward(new)
                
                if(self.creation_op == "transpose"):
                    self.creators[0].backward(self.grad.transpose())
                    
                if("sum" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    ds = self.creators[0].data.shape[dim]
                    self.creators[0].backward(self.grad.expand(dim, ds))
                
                if("expand" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.sum(dim))
                    
                if(self.creation_op == "neg"):
                    self.creators[0].backward(self.grad.__neg__())

    def __add__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                           autograd=True,
                           creators=[self,other],
                           creation_op="add")
        return Tensor(self.data + other.data)
    
    def __neg__(self):
        if(self.autograd):
            return Tensor(self.data * -1,
                           autograd=True,
                           creators=[self],
                           creation_op="neg")
        return Tensor(self.data * -1)
    
    def __sub__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data - other.data,
                           autograd=True, creators=[self, other],
                           creation_op="sub")
        return Tensor(self.data - other.data)
    
    def __mul__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data * other.dta,
                           autograd=True, creators=[self, other],
                           creation_op="mul")
        return Tensor(self.data * other.data)
    
    def sum(self, dim):
        if(self.autograd):
            return Tensor(self.data.sum(dim),
                           autograd=True, creators=[self],
                           creation_op="sum_" + str(dim))
        return Tensor(self.data.sum(dim))
    
    def expand(self, dim, copies):
        trans_cmd = list(range(0, len(self.data.shape)))
        trans_cmd.insert(dim, len(self.data.shape))
        new_shape = list(self.data.shape) + [copies]
        new_data = \
            self.data.repeat(copies).reshape(list(self.data.shape) + [copies])
        new_data = new_data.transpose(trans_cmd)
        if(self.autograd):
            return Tensor(new_data,
                           autograd=True, creators=[self],
                           creation_op="expand_" + str(dim))
        return Tensor(new_data)
    
    def transpose(self):
        if(self.autograd):
            return Tensor(self.data.transpose(),
                           autograd=True, creators=[self],
                           creation_op="transpose")
        return Tensor(self.data.transpose())
    
    def mm(self, x):
        if(self.autograd):
            return Tensor(self.data.dot(x.data),
                           autograd=True, creators=[self, x],
                           creation_op="mm")
        return Tensor(self.data.dot(x.data))
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())

a = Tensor([1,2,3,4,5], autograd=True)
b = Tensor([2,2,2,2,2], autograd=True)
c = Tensor([5,4,3,2,1], autograd=True)

d = a + b
e = b + c
f = d + e

f.backward(Tensor(np.array([1,1,1,1,1])))

print(b.grad.data == np.array([2,2,2,2,2]))

[ True  True  True  True  True]


In [6]:
x = Tensor(np.array([[1,2,3], [4,5,6]]))

In [7]:
x.sum(0)

array([5, 7, 9])

In [8]:
x.sum(1)

array([ 6, 15])

In [9]:
x.expand(dim=0, copies=4)

array([[[1, 2, 3],
        [4, 5, 6]],

       [[1, 2, 3],
        [4, 5, 6]],

       [[1, 2, 3],
        [4, 5, 6]],

       [[1, 2, 3],
        [4, 5, 6]]])

In [10]:
import numpy

np.random.seed(0)
data = np.array([[0,0],[0,1],[1,0],[1,1]])
target = np.array([[0],[1],[0],[1]])

weights_0_1 = np.random.rand(2,3)
weights_1_2 = np.random.rand(3,1)

for i in range(10):
    # 予測
    layer_1 = data.dot(weights_0_1)
    layer_2 = layer_1.dot(weights_1_2)
    
    # 比較
    diff = (layer_2 - target)
    sqdiff = (diff * diff)
    loss = sqdiff.sum(0) # 損失値(平均二乗誤差)
    
    # 学習の逆伝播部分
    layer_1_grad = diff.dot(weights_1_2.transpose())
    weight_1_2_update = layer_1.transpose().dot(diff)
    weight_0_1_update = data.transpose().dot(layer_1_grad)
    
    weights_1_2 -= weight_1_2_update * 0.1
    weights_0_1 -= weight_0_1_update * 0.1
    print(loss[0])

5.066439994622395
0.4959907791902342
0.4180671892167177
0.35298133007809646
0.2972549636567377
0.2492326038163328
0.20785392075862477
0.17231260916265176
0.14193744536652986
0.11613979792168384
